# HMC Case: Mean-Variance Optimization Analysis

In [23]:
import pandas as pd
import math
import numpy as np

In [34]:
excess_returns = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='excess returns').set_index('Date')
excess_returns = excess_returns.drop('QAI',axis=1)


# 2.1 Summary Statistics

In [ ]:

excess_mean = excess_returns.mean() * 12
excess_vol = excess_returns.std() * math.sqrt(12)
excess_sharpe = excess_mean / excess_vol

print(pd.DataFrame({'Mean' : excess_mean,'Volatility': excess_vol, 'Sharpe': excess_sharpe}))


         Mean  Volatility    Sharpe
BWX -0.007716    0.082789 -0.093202
DBC -0.005292    0.166553 -0.031774
EEM  0.029339    0.176164  0.166542
EFA  0.061775    0.150903  0.409372
HYG  0.041371    0.075928  0.544873
IEF  0.016404    0.063442  0.258569
IYR  0.074916    0.168675  0.444143
PSP  0.092561    0.213370  0.433804
SPY  0.128141    0.142839  0.897103
TIP  0.020502    0.051115  0.401091


##### As we can see, SPY has the best sharpe ratio and BWX has the worst sharpe ratio

# 2.2 Descriptive Analysis

In [ ]:
correlation_matrix = excess_returns.corr()
# Unstack the matrix to get a series of all pairs, then drop self-correlations
corr_pairs = correlation_matrix.unstack()
corr_pairs = corr_pairs.drop_duplicates() # Drop duplicate pairs
corr_pairs = corr_pairs[corr_pairs.abs() != 1] # Drop self-correlations

highest_corr_pair = corr_pairs.idxmax()
lowest_corr_pair = corr_pairs.idxmin()

print("Correlation Matrix:")
display(correlation_matrix.style.background_gradient(cmap='coolwarm').format("{:.2f}"))

print(f"\nPair with the highest correlation: {highest_corr_pair} ({correlation_matrix.loc[highest_corr_pair]:.4f})")
print(f"Pair with the lowest correlation: {lowest_corr_pair} ({correlation_matrix.loc[lowest_corr_pair]:.4f})")

Correlation Matrix:


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,SPY,TIP
BWX,1.00,0.19,0.62,0.60,0.60,0.58,0.55,0.53,0.44,0.68
DBC,0.19,1.00,0.51,0.50,0.46,-0.30,0.28,0.45,0.43,0.11
EEM,0.62,0.51,1.00,0.82,0.69,0.03,0.58,0.75,0.69,0.38
EFA,0.60,0.50,0.82,1.00,0.79,0.04,0.70,0.90,0.85,0.39
HYG,0.60,0.46,0.69,0.79,1.00,0.19,0.74,0.81,0.79,0.54
IEF,0.58,-0.30,0.03,0.04,0.19,1.00,0.32,0.02,0.00,0.75
IYR,0.55,0.28,0.58,0.70,0.74,0.32,1.00,0.75,0.75,0.60
PSP,0.53,0.45,0.75,0.90,0.81,0.02,0.75,1.00,0.89,0.41
SPY,0.44,0.43,0.69,0.85,0.79,0.00,0.75,0.89,1.00,0.38
TIP,0.68,0.11,0.38,0.39,0.54,0.75,0.60,0.41,0.38,1.00



Pair with the highest correlation: ('EFA', 'PSP') (0.8953)
Pair with the lowest correlation: ('DBC', 'IEF') (-0.3002)


TIPS vs IEF vs BWX

 
 * **TIPS vs. Domestic Bonds (IEF):** TIPS have a higher mean return, slightly higher volatility, and a significantly better Sharpe ratio than IEF. This suggests they have offered better risk-adjusted performance than domestic bonds in this sample.
 * **TIPS vs. Foreign Bonds (BWX):** TIPS also outperform foreign bonds, showing a higher mean return, lower volatility, and a much higher Sharpe ratio.
 
 Overall, TIPS have performed well relative to other major bond categories in this dataset.


# Section 2.3: The MV Frontier & Tangency Portfolio

Calculating the tangency weights

In [ ]:
cov_matrix = excess_returns.cov() * 12
inv_cov_matrix = np.linalg.inv(cov_matrix)
tangency_weights_unscaled = inv_cov_matrix @ excess_mean
tangency_weights = tangency_weights_unscaled / tangency_weights_unscaled.sum()

tangency_weights_df = pd.DataFrame(tangency_weights, index=excess_returns.columns, columns=['Weight'])
print("Tangency Portfolio Weights (w_T):")
display(tangency_weights_df.style.format("{:.2%}"))


Tangency Portfolio Weights (w_T):


,Weight
BWX,-85.06%
DBC,-7.16%
EEM,2.64%
EFA,6.87%
HYG,29.06%
IEF,88.12%
IYR,-24.66%
PSP,-33.30%
SPY,105.96%
TIP,17.53%


Tangency Portfolio Performance

In [32]:
tangency_return = tangency_weights @ excess_mean
tangency_volatility = np.sqrt(tangency_weights.T @ cov_matrix @ tangency_weights)
tangency_sharpe = tangency_return / tangency_volatility

print("\nTangency Portfolio Performance (Annualized):")
print(f"Mean: {tangency_return:.4f}")
print(f"Volatility: {tangency_volatility:.4f}")
print(f"Sharpe Ratio: {tangency_sharpe:.4f}")


Tangency Portfolio Performance (Annualized):
Mean: 0.1285
Volatility: 0.0875
Sharpe Ratio: 1.4692


# Section 2.4: TIPS Sensitivity Analysis

Analysis without TIPS

In [37]:
excess_returns_no_tips = excess_returns.drop(columns=['TIP'])
mean_returns_no_tips = excess_returns_no_tips.mean() * 12
cov_matrix_no_tips = excess_returns_no_tips.cov() * 12
inv_cov_no_tips = np.linalg.inv(cov_matrix_no_tips)
weights_unscaled_no_tips = inv_cov_no_tips @ mean_returns_no_tips
no_tips_weights = weights_unscaled_no_tips / weights_unscaled_no_tips.sum()

no_tips_return = no_tips_weights @ mean_returns_no_tips
no_tips_volatility = np.sqrt(no_tips_weights.T @ cov_matrix_no_tips @ no_tips_weights)
no_tips_sharpe = no_tips_return / no_tips_volatility

Analysis with Adjusted TIPS Return

In [36]:
annual_return_increase = 0.0012 * 12
mean_returns_adj = excess_mean.copy()
mean_returns_adj['TIP'] += annual_return_increase

weights_unscaled_adj = inv_cov_matrix @ mean_returns_adj
adj_tips_weights = weights_unscaled_adj / weights_unscaled_adj.sum()

adj_tips_return = adj_tips_weights @ mean_returns_adj
adj_tips_volatility = np.sqrt(adj_tips_weights.T @ cov_matrix @ adj_tips_weights)
adj_tips_sharpe = adj_tips_return / adj_tips_volatility

Comparison of both the Adjusted TIPS Return and without TIPS

In [38]:
comparison_df = pd.DataFrame({
    'Annualized Mean': [tangency_return, no_tips_return, adj_tips_return],
    'Annualized Volatility': [tangency_volatility, no_tips_volatility, adj_tips_volatility],
    'Annualized Sharpe Ratio': [tangency_sharpe, no_tips_sharpe, adj_tips_sharpe]
}, index=['Base Case (with TIPS)', 'No TIPS', 'Adjusted TIPS Return'])

print("Performance Comparison Across Scenarios:")
display(comparison_df.style.format("{:.4f}"))

print("\nWeight of TIPS in each scenario:")
print(f"Base Case: {tangency_weights_df.loc['TIP', 'Weight']:.2%}")
print(f"Adjusted TIPS Return: {adj_tips_weights[excess_returns.columns.get_loc('TIP')]:.2%}")


Performance Comparison Across Scenarios:


,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
Base Case (with TIPS),0.1285,0.0875,1.4692
No TIPS,0.1327,0.0905,1.4675
Adjusted TIPS Return,0.1204,0.0746,1.6124



Weight of TIPS in each scenario:
Base Case: 17.53%
Adjusted TIPS Return: 128.25%


Yes absolutely, the analysis clearly shows that including TIPS expands the investment opportunity set. 
If TIPS is removed then the sharpe ratio decreases, and if we give it a bit boost then we actually improve the sharpe by a lot.

# Section 3: Allocations

Now we will construct three different portfolios—Mean-Variance (MV), Equally-Weighted (EW), and Risk-Parity (RP)—each scaled to have a targeted mean monthly excess return of 1%. We will then compare their performance.

In [39]:
mu_target_monthly = 0.01
mu_target_annual = mu_target_monthly * 12

monthly_mean_returns = excess_returns.mean()
monthly_cov_matrix = excess_returns.cov()

1. Equally weighted (EW) Portfolio

In [40]:
n_assets = len(excess_returns.columns)
ew_weights_initial = np.ones(n_assets) / n_assets
ew_mean_monthly = ew_weights_initial @ monthly_mean_returns
# Scale the weights to meet the target
ew_scalar = mu_target_monthly / ew_mean_monthly
ew_weights = ew_scalar * ew_weights_initial

2. Risk-Parity (RP) Portfolio

In [41]:
asset_variances = np.diag(monthly_cov_matrix)
rp_weights_initial = (1 / asset_variances) / sum(1 / asset_variances)
rp_mean_monthly = rp_weights_initial @ monthly_mean_returns
# Scale the weights to meet the target
rp_scalar = mu_target_monthly / rp_mean_monthly
rp_weights = rp_scalar * rp_weights_initial

3. Mean-Variance (MV) Portfolio

In [42]:
w_T = mu_target_monthly / (tangency_weights @ monthly_mean_returns)
mv_weights = w_T * tangency_weights

#### Performance Comparison

In [44]:
portfolios = {'MV': mv_weights, 'EW': ew_weights, 'RP': rp_weights}
performance_data = {}

for name, weights in portfolios.items():
    ann_mean = (weights @ monthly_mean_returns) * 12
    ann_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe = ann_mean / ann_vol
    performance_data[name] = {'Annualized Mean': ann_mean, 'Annualized Volatility': ann_vol, 'Annualized Sharpe Ratio': sharpe}

allocation_comparison_df = pd.DataFrame(performance_data).T
print("Performance of Scaled Allocation Strategies:")
display(allocation_comparison_df.style.format("{:.4f}"))

Performance of Scaled Allocation Strategies:


,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
MV,0.1200,0.0817,1.4692
EW,0.1200,0.2692,0.4457
RP,0.1200,0.2639,0.4547


the Mean-Variance (MV) portfolio is superior. It achieves the target return with the lowest volatility, and has the highest Sharpe ratio.